In [1]:
import numpy as np

In [2]:
# image to column : 이미지를 입력받으면 2차원 배열로 전개한다.  

# input_data : 4차원 shape의 입력 데이터(이미지 수, 채널 수, 높이, 너비)
# filter_h : 필터의 높이
# filter_w : 필터의 너비
# stride : 스트라이드
# pad : 패딩
# out_h : 출력 데이터의 높이
# out_w : 출력 데이너의 너비

def im2col(input_data, filter_h, filter_w, stride=1, pad=0):

    N, C, H, W = input_data.shape
    out_h = (H + 2*pad - filter_h) // stride + 1
    out_w = (W + 2*pad - filter_w) // stride + 1

    img = np.pad(input_data, [(0,0), (0,0), (pad, pad), (pad, pad)], 'constant')
    col = np.zeros((N, C, filter_h, filter_w, out_h, out_w))

    for y in range(filter_h):
        y_max = y + stride * out_h
        for x in range(filter_w):
            x_max = x + stride * out_w
            col[:, :, y, x, :, :] = img[:, :, y:y_max:stride, x:x_max:stride]

    col = col.transpose(0, 4, 5, 1, 2, 3).reshape(N * out_h * out_w, -1)
    
    # col은 2차원 배열
    return col

In [3]:
# column to image : 2차원 배열을 입력받아 다수의 이미지 묶음으로 변환한다. 

#  col : 2차원 배열(입력 데이터)
# input_shape : 원래 이미지 데이터의 shape, (데이터 수, 채널 수, 높이, 너비) 의 4차원
# filter_h : 필터의 높이
# filter_w : 필터의 너비
# stride : 스트라이드
# pad : 패딩
# img : 변환된 이미지 묶음

def col2im(col, input_shape, filter_h, filter_w, stride=1, pad=0):
    
    N, C, H, W = input_shape
    out_h = (H + 2*pad - filter_h)//stride + 1
    out_w = (W + 2*pad - filter_w)//stride + 1
    col = col.reshape(N, out_h, out_w, C, filter_h, filter_w).transpose(0, 3, 4, 5, 1, 2)

    img = np.zeros((N, C, H + 2*pad + stride - 1, W + 2*pad + stride - 1))
    for y in range(filter_h):
        y_max = y + stride*out_h
        for x in range(filter_w):
            x_max = x + stride*out_w
            img[:, :, y:y_max:stride, x:x_max:stride] += col[:, :, y, x, :, :]

    return img[:, :, pad:H + pad, pad:W + pad]


In [4]:
# 합성곱 계층은 필터(W), 편향, 스트라이드, 패딩을 인수로 받아 초기화한다.
# 필터는 (FN_필터의 개수, C_채널, FH_필터의 높이, FW_필터의 너비)의 4차원 shape

class Convolution:
    def __init__(self, W, b, stride=1, pad=0):
        self.W = W
        self.b = b
        self.stride = stride
        self.pad = pad
    
    def forward(self, x):
        FN, C, FH, FW = self.W.shape
        N, C, H, W = x.shape
        out_h = int(( (H + 2*self.pad - FH) / self.stride) + 1)
        out_w = int(( (W + 2*self.pad - FW) / self.stride) + 1)
        
        # 입력데이터를 im2col로 전개하고, 필터도 reshape로 2차원 배열로 전개한다.
        col = im2col(x, FH, FW, self.stride, self.pad)
        col_w = self.W.reshape(FN, -1).T
        out = np.dot(col, col_W) + self.b
        
        out = out.reshape(N, out_h, out_w, -1).transpose(0, 3, 1, 2)
        
        return out
    
    # 역전파는 Affine 계층의 역전파와 비슷하다. 전치 대신 col2im을 사용한다는 점이 차이점.
    def backward(self, dout):
        FN, C, FH, FW = self.W.shape
        dout = dout.transpose(0,2,3,1).reshape(-1, FN)

        self.db = np.sum(dout, axis=0)
        self.dW = np.dot(self.col.T, dout)
        self.dW = self.dW.transpose(1, 0).reshape(FN, C, FH, FW)

        dcol = np.dot(dout, self.col_W.T)
        dx = col2im(dcol, self.x.shape, FH, FW, self.stride, self.pad)

        return dx